# Introduction to Non Linearity

In [40]:
require 'nn'
require 'gnuplot'
require 'tools' 
Plot = require 'itorch.Plot'

## Context

* **Master DAC** Machine Learning and Deep Learning
* **Students** : Youcef Benyettou & David Panou


This notebook follows the instructions given in _"Non-Linearity Assignment"_ written by **Prof. Denoyer** and **Prof. Baskiotis** at UPMC.
Its intent is to demonstrate the expression capacity of simple neural networks for nonlinear problems.

# Linear problems

Linear problems are problems in which a **linear model** linear models implementation can provide a good solution, or a good approximation to it.

Those specific kind of datasets are thus called **linearly separable** and we go in depth of such problems in our previous [work](../tme3/Mini-Batch - Train-Test Criterion implementation - TME2.ipynb)

In this part, we will go through an example of well-known non-linear problem - The XOR problem - and analyze how neural networks perform in this kind of issues.

# The XOR problem statement

Compared to linear problem, the XOR problem is a somehow a bit more cumbersome to solve for linear models since there is no linear models that can achieve good results on it.

In [23]:
function create_xor_problem()
    x = torch.Tensor(200,2)
    y = torch.Tensor(200,1)
    
    dataset = {}
    
    for i=1,50 do
            x[i]=torch.randn(2)
            x[i][1]=x[i][1]+3
            x[i][2]=x[i][2]+3
            y[i]=torch.Tensor(1):fill(1)
    end
    for i=51,100 do
            x[i]=torch.randn(2)
            x[i][1]=x[i][1]-3
            x[i][2]=x[i][2]-3
            y[i]=torch.Tensor(1):fill(1)
    end
    for i=101,150 do
            x[i]=torch.randn(2)
            x[i][1]=x[i][1]-3
            x[i][2]=x[i][2]+3
            y[i]=torch.Tensor(1):fill(-1)
    end
    for i=151,200 do
            x[i]=torch.randn(2)
            x[i][1]=x[i][1]+3
            x[i][2]=x[i][2]-3
            y[i]=torch.Tensor(1):fill(-1)
    end
    
    dataset["data"] = x
    dataset["labels"] = y
    
    setmetatable(dataset,
    {__index = function(t, i)
                return {t.data[i], t.label[i]}
               end})

    function dataset:size()
        return self.data:size()
    end
    
    return dataset
end

In [24]:
dataset = create_xor_problem()

## Prepare the dataset to representation

In [68]:
function create_categorical_set(xs)
    cat_1 = {}
    cat_1["x"] = {}
    cat_1["y"] = {}

    cat_2 = {}
    cat_2["x"] = {}
    cat_2["y"] = {}

    l = 1
    j = 1
    
    ys = xs["labels"]

    for i=1,xs:size()[1] do
        if ys[i][1]==1 then
            cat_1["x"][l]= xs["data"][i][1]
            cat_1["y"][l]= xs["data"][i][2]
            l = l+1
        else
            cat_2["x"][j]= xs["data"][i][1]
            cat_2["y"][j]= xs["data"][i][2]
            j=j+1
        end
    end
    
    return {cat_1,cat_2}
end

In [69]:
categories = create_categorical_set(dataset)

## XOR problem representation

### Using iTorch

In [71]:
-- scatter plots 
plot = Plot():circle(categories[1]["x"],categories[1]["y"],'red','accuracy'):circle(categories[2]["x"],categories[2]["y"],'blue','category_2'):draw()
plot:title('XOR Problem Representation'):redraw() 
--plot:xaxis('iteration'):yaxis('MSE - Accuracy'):redraw() 
--plot:legend(true) 
plot:save('out.html')

### Using gnuplot

```lua
gnuplot.figure()
gnuplot.plot({x[{{1,100}}],'+' },{x[{{101,200}}],'+'})
```

We are using the previously used 

In [ ]:
function accuracy(labels,output)
    -- This method computes the accuracy of a set of prediction y, compared to some expected labels
    local pred_signs = torch.sign(output)
    local correct_classification = torch.cmul(pred_signs,labels)
    --local eval_params = torch.Tensor(1):fill(1):double()
    return torch.mean(correct_classification:eq(1):double())
end

# Creating the Neural Network

The Neural Network we are about to create will be a 2->5->1 neural network.

In [90]:
-- 2 : creation du modele

model1= nn.Linear(2,5)
model2= nn.Linear(5,1)

criterion= nn.MSECriterion() 
tanh = nn.Tanh()
tanh1 = nn.Tanh()
tanh2 = nn.Tanh()

output1=torch.Tensor(5):fill(0) --o1
y1=torch.Tensor(5):fill(0) --o2
y2=torch.Tensor(1):fill(0) --o4
output2=torch.Tensor(1):fill(0) --o3
delta1 = 0
delta2 = 0
delta1tanh = 0
delta2tanh = 0
idx = 0


-- 3 : Boucle d'apprentissage
learning_rate= 0.01 
maxEpoch= 500
loss = 0
all_losses={}


for iteration=1,maxEpoch do
	model1:zeroGradParameters()
        model2:zeroGradParameters()
	idx=math.random(x:size(1))
	--forward
	output1 = model1:forward(x[idx])
    	y1 = tanh1:forward(output1)
	output2 = model2:forward(y1)
   	y2 = tanh2:forward(output2) 
	loss = loss + criterion:forward(y2,y[idx])

	--backward
	delta2 = criterion:backward(y2,y[idx])
	delta2tanh = tanh2:backward(output2,delta2)
	delta1 = model2:backward(y1,delta2tanh)
	model2:updateParameters(learning_rate)	
	delta1tanh = tanh1:backward(output1,delta1)
	model1:backward(x[idx],delta1tanh)
	model1:updateParameters(learning_rate)
	
end	
	

out1 = model1:forward(x)
out2 = tanh1:forward(out1)
out3 = model2:forward(out2)
pred = tanh2:forward(out3)
acc = accuracy(y,pred)
print('Precision : '..acc)




Precision : 0.97	
